# OWL tutorial

First install owlready2 if you don't already have it, and have a quick look at the [documentation](https://owlready2.readthedocs.io/en/v0.36/onto.html#).

In [1]:
## Uncomment if you do not have owlrl installed (you should have it installed from the RDFS tutorial)
#import sys
#!{sys.executable} -m pip install rdflib  owlready2 pandas

import pandas as pd
from rdflib import Graph, Literal, Namespace, RDF, URIRef, OWL
from rdflib.namespace import DC, FOAF

from owlready2 import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Let's start loading some data from a .CSV file. We are going to create an ontology that describes the data inside.
We already did part of this using the semantics of RDF(S), now we'll use the semantics of [OWL](https://www.w3.org/TR/2012/REC-owl2-primer-20121211/) through owlready2. 

Remember that an ontology is often an application ontology, meaning that it is built with a specific task in mind. 
We could model _everything_ within a certain domain in the most ontologically correct way possible, _or_ **we could model the domain in accordance with the application's task.** 


**Your task and domain:** You are a broadcaster that has just digitised its radio archives into a digital music archive (DMA), and aims to play more interesting tracks by discovering their 'hidden treasures', by making unexpected and potentially interesting relations between tracks visible to the users (which are journalists and program makers).


**Exercise 1** 

1. look at the .csv files in the folder /data/musicoset_metadata/ and load them into pandas dataframes (use display.max_columns to show all columns). 
2. initialise an empty ontology using owlready2
3. using owlready2, create a hierarchy of classes and subclasses that describe the entities in your dataframes
4. using owrleady2, create properties and subproperties their properties, and how the classes relate to one another (using domain and range). If it helps: draw out your ontology in https://app.diagrams.net/
    - create: object properties, data properties, functional properties
5. using owlready2, add class restrictions
6. create invididuals of your classes, and provide them with attributes using your properties! 
7. write simple queries to retrieve your individuals following: https://owlready2.readthedocs.io/en/v0.36/onto.html#simple-queries. What kind of things would journalists and program makers like to retrieve? 
6. save your asserted owl file

In [2]:
#load each csv in a different df
df_albums = pd.read_csv('./data/musicoset_metadata/albums.csv', sep='\t')
df_artists = pd.read_csv('./data/musicoset_metadata/artists.csv',sep='\t')
df_releases = pd.read_csv('./data/musicoset_metadata/releases.csv',sep='\t')
df_songs = pd.read_csv('./data/musicoset_metadata/songs.csv',sep='\t')
df_tracks = pd.read_csv('./data/musicoset_metadata/tracks.csv',sep='\t')


df_albums.head()


,album_id,name,billboard,artists,popularity,total_tracks,album_type,image_url
0,5n1GSzC1Reao29ScnpLYqp,Dying To Live,Dying To Live,{'46SHBwWsqBkxI7EeeBEQG7': 'Kodak Black'},83,16,album,https://i.scdn.co/image/db2133234d458f432ca207...
1,6UYZEYjpN1DYRW0kqFy9ZE,Championships,Championships,{'20sxb77xiYeusSH8cVdatc': 'Meek Mill'},85,19,album,https://i.scdn.co/image/77eb7c17cafe5503c58661...
2,7uVimUILdzSZG4KKKWToq0,Christmas (Deluxe Special Edition),Christmas,{'1GxkXlMwML1oSg5eLPiAz3': 'Michael Bublé'},60,20,album,https://i.scdn.co/image/2d6ee8d4fb5a45abf35cd3...
3,35s58BRTGAEWztPo9WqCIs,Spider-Man: Into the Spider-Verse (Soundtrack ...,Spider-Man: Into The Spider-Verse,{'0LyfQWJT6nXafLPZqxe9Of': 'Various Artists'},92,13,compilation,https://i.scdn.co/image/3aa37254a41cf96e815725...
4,41GuZcammIkupMPKH2OJ6I,ASTROWORLD,ASTROWORLD,{'0Y5tJX1MQlPlqiwlOH1tJY': 'Travis Scott'},91,17,album,https://i.scdn.co/image/cdca7dc20c778ada42fb18...


In [3]:

#initialize empty ontology

onto = get_ontology("http://test.org/music.owl") 

In [4]:
with onto:
    class Person(Thing):
        pass
    
    class Song(Thing):
        pass
    
    class releasDate(DataProperty, FunctionalProperty):
        domain    = [Song]
        range     = [str]
        pass
    
    class Album(Thing):
        pass
    
    class Artist(Thing):
        pass
    
    class hasSong(ObjectProperty):
        domain    = [Album]
        range     = [Song]
        pass
    
    class writtenBy(ObjectProperty):
        domain    = [Song]
        range     = [Artist]
        pass
    
    class performedBy(Song >> Artist):
        pass
        
    class bandMember(Artist >> Artist):
        pass
    
    class name(DataProperty, FunctionalProperty):
        range     = [str]
        pass
    
#     class hasName(ObjectProperty):
#         domain    = [Thing]
#         range     = [Name]

    class Billboard(Thing):
        pass
    
    class Popularity(DataProperty, FunctionalProperty):
        range    = [int]
        pass
    
    class ImageUrl(Thing):
        pass
    
    class Genre(Thing):
        pass
    
    class SubGenre(Genre):
        pass
    
    class hasSubGenre(Genre >> SubGenre):
        pass
    
    class hasGenre(ObjectProperty):
        domain   = [Song]
        range    = [Genre]
        pass
    
    class ArtistType(Thing):
        pass
        
    class hasType(Artist >> ArtistType):
        pass
    
    
    
#Create classes - each class is subclass of thing

In [5]:
#Create properties and subproperties
list(onto.classes())

[music.Person,
 music.Song,
 music.Album,
 music.Artist,
 music.Billboard,
 music.ImageUrl,
 music.Genre,
 music.SubGenre,
 music.ArtistType]

In [6]:
#add class restrictions
Song.is_a.append(writtenBy.min(1))
Song.is_a.append(performedBy.min(1))
Song.is_a.append(releasDate.max(1))
Song.is_a.append(hasGenre.min(1))

Person.is_a.append(name.exactly(1))
Album.is_a.append(name.exactly(1))
Song.is_a.append(name.exactly(1))

Artist.is_a.append(hasType.min(1))
Album.is_a.append(hasSong.min(1))

class SoloArtist(Artist):
    equivalent_to = [Artist & bandMember.max(1)]
    namespace=onto
    pass

class Band(Artist):
    equivalent_to = [Artist & bandMember.min(2)]
    namespace=onto
    pass

list(onto.properties())

[music.releasDate,
 music.name,
 music.Popularity,
 music.hasSong,
 music.writtenBy,
 music.performedBy,
 music.bandMember,
 music.hasSubGenre,
 music.hasGenre,
 music.hasType]

In [7]:
#create instances of the classes
alice = Person(name='Alice')
alice.is_a.append(Artist)
bob = Person(name='Bob')
bob.is_a.append(Artist)
chris = Person(name='Chris')
chris.is_a.append(Artist)

banger_boys = Artist(bandMember=[alice, bob, chris], name='Banger Boys')

heart = Song(writtenBy=[bob], performedBy=[banger_boys], releaseDate='2-0-2020', name='My heart goes BOOM')

head = Song(writtenBy=[alice], performedBy=[banger_boys], releaseDate='2-1-2021', name='My head goes WHOAHH')

feet = Song(writtenBy=[chris], performedBy=[banger_boys], releaseDate='2-2-2022', name='My feet go WILD')

bootleg_billionaire = Album(hasSong=[heart, head, feet], name='Bootleg Billionaire')

pop     = Genre('pop')
rock    = Genre('rock')
triphop = Genre('triphop')
experimentalRock = SubGenre('experimentalRock')
latin   = Genre('latin')
salsa   = SubGenre('salsa')
reggeaton = SubGenre('reggeaton')

latin.hasSubGenre = [salsa, reggeaton]
rock.hasSubGenre = [experimentalRock]

heart.hasGenre = [triphop]
head.hasGenre = [experimentalRock]
feet.hasGenre = [reggeaton, experimentalRock]

AllDifferent([pop, rock, triphop, latin])

AllDisjoint([music.pop, music.rock, music.triphop, music.latin])

In [8]:
#write simple queries HINT (onto.search, ..)
onto.search(writtenBy='*')

[music.My heart goes BOOM, music.My head goes WHOAHH, music.My feet go WILD]

In [9]:
#save
onto.save(file = "./data/musicOnto.owl", format = "rdfxml")


## OWL reasoning 

Let's look at how reasoning works.

Owlready automatically gets the results of the reasoning from HermiT (a type of reasoner) and reclassifies Individuals and Classes. 

**Exercise 2**
1. think about which things are inferred from your OWL semantics. Query/look at your graph: do you see what you expected?
2. looking at the following tutorial [owlready2-reasoning](https://owlready2.readthedocs.io/en/latest/reasoning.html), which things have not yet been inferred? Run the owlready2 reasoner to:
    - infer these new triples
    - check your ontology and statements (individuals + attributes) for consistency
3. save your asserted + inferred triples to a new file 

In [10]:

print("Persons:")
for p in Person.instances():
    print(p)
    
print('\nArtists:')
for a in Artist.instances():
    print(a)

    
# This is working!
print('\nGenre:')
for g in Genre.instances():
    print(g)

Persons:
music.Alice
music.Bob
music.Chris

Artists:
music.Alice
music.Bob
music.Chris
music.Banger Boys

Genre:
music.pop
music.rock
music.triphop
music.latin
music.experimentalRock
music.salsa
music.reggeaton


In [11]:
new_project = Album(name='triple trillionaires')
print("Albums:")
for a in Album.instances():
    print(a)
    

with onto:
    sync_reasoner(infer_property_values = True)

# This is not!!!!!    
print('Checking of inconsitencies')

list(onto.inconsistent_classes()) # should show the inconsistancy of 'music.triple trillionaires' not having any songs

print('Bands:')
for b in Band.instances(): # It should be inferred that 'music.banger boys' is a 'music.Band'
    print(b)

print('Solo Artists:') # It should be inferred that 'music.alice', 'music.bob', and 'music.chris' are all solo artists
for s in SoloArtist.instances():
    print(s)

print('\nHere the inferred classes should show up as well:') # basically different ways to check the same things mentioned above
print(onto.get_parents_of(bob))
print(banger_boys.is_a)

Albums:
music.Bootleg Billionaire
music.triple trillionaires


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/user/.pyenv/versions/3.9.2/envs/KRW-3.9.2/lib/python3.9/site-packages/owlready2/hermit:/home/user/.pyenv/versions/3.9.2/envs/KRW-3.9.2/lib/python3.9/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmp3_sgadnq -Y


Checking of inconsitencies
Bands:
Solo Artists:

Here the inferred classes should show up as well:
[music.Person, music.Artist]
[music.Artist]


* Owlready2 * HermiT took 0.3146069049835205 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


#### Querying inferred triples

**Exercise 3**
Query your inferred triples: 

- *.get_parents_of(entity)* accepts any entity (Class, property or individual), and returns the superclasses (for a class), the superproperties (for a property), or the classes (for an individual). 

- *.get_instances_of(Class)* returns the individuals that are asserted as belonging to the given Class in the ontology. (NB for obtaining all instances, independently of the ontology they are asserted in, use Class.instances()).

- *.get_children_of(entity)* returns the subclasses (or subproperties) that are asserted for the given Class or property in the ontology. (NB for obtaining all children, independently of the ontology they are asserted in, use entity.subclasses()).